In [1]:
import pandas as pd
import numpy as np

import re
import os

In [3]:
from ddf_utils.str import to_concept_id, format_float_digits
from ddf_utils.index import create_index_file

In [5]:
def read_source(f, skip=0, **kwargs):
    df = pd.read_csv(f, **kwargs)
    # quick fix for malformed csv downloaded from data povider
    if df.columns[0] == 'Year"':
        df = df.rename(columns={'Year"': 'Year'})
    df.columns = list(map(lambda x: x.lower().replace('\n', ''), df.columns))
    df = df.ix[skip:]  # skip first few rows of data
    
    return df

In [9]:
nation_file = '../source/nation.1751_2013.csv'
global_file = '../source/global.1751_2013.csv'

In [10]:
nation_data = read_source(nation_file, skip=3, na_values='.')
global_data = read_source(global_file, skip=1, na_values='.')

In [13]:
global_data.head()

,year,total carbon emissions from fossil fuel consumption and cement production (million metric tons of c),carbon emissions from gas fuel consumption,carbon emissions from liquid fuel consumption,carbon emissions from solid fuel consumption,carbon emissions from cement production,carbon emissions from gas flaring,per capita carbon emissions (metric tons of carbon; after 1949 only)
1,1751,3,0,0.0,3.0,0.0,0.0,NaN
2,1752,3,0,0.0,3.0,0.0,0.0,NaN
3,1753,3,0,0.0,3.0,0.0,0.0,NaN
4,1754,3,0,0.0,3.0,0.0,0.0,NaN
5,1755,3,0,0.0,3.0,0.0,0.0,NaN


In [14]:
nation_data.head()

,nation,year,total co2 emissions from fossil-fuels and cement production (thousand metric tons of c),emissions from solid fuel consumption,emissions from liquid fuel consumption,emissions from gas fuel consumption,emissions from cement production,emissions from gas flaring,per capita co2 emissions (metric tons of carbon),emissions from bunker fuels (not included in the totals)
3,AFGHANISTAN,1949.0,4.0,4.0,0.0,0.0,0.0,NaN,NaN,0.0
4,AFGHANISTAN,1950.0,23.0,6.0,18.0,0.0,0.0,0.0,0.0,0.0
5,AFGHANISTAN,1951.0,25.0,7.0,18.0,0.0,0.0,0.0,0.0,0.0
6,AFGHANISTAN,1952.0,25.0,9.0,17.0,0.0,0.0,0.0,0.0,0.0
7,AFGHANISTAN,1953.0,29.0,10.0,18.0,0.0,0.0,0.0,0.0,0.0


In [15]:
nation_data.year.hasnans

False

In [16]:
# fix year to int
nation_data.year = nation_data.year.map(int)
global_data.year = global_data.year.map(int)

In [17]:
nation_data.head()

,nation,year,total co2 emissions from fossil-fuels and cement production (thousand metric tons of c),emissions from solid fuel consumption,emissions from liquid fuel consumption,emissions from gas fuel consumption,emissions from cement production,emissions from gas flaring,per capita co2 emissions (metric tons of carbon),emissions from bunker fuels (not included in the totals)
3,AFGHANISTAN,1949,4.0,4.0,0.0,0.0,0.0,NaN,NaN,0.0
4,AFGHANISTAN,1950,23.0,6.0,18.0,0.0,0.0,0.0,0.0,0.0
5,AFGHANISTAN,1951,25.0,7.0,18.0,0.0,0.0,0.0,0.0,0.0
6,AFGHANISTAN,1952,25.0,9.0,17.0,0.0,0.0,0.0,0.0,0.0
7,AFGHANISTAN,1953,29.0,10.0,18.0,0.0,0.0,0.0,0.0,0.0


# concepts

We are going to make the concept names for measures simpler. See https://github.com/open-numbers/ddf--cdiac--co2/issues/3

In [45]:
def get_concept_id(name):
    """return concept name for given indicator name.
    """
    if 'total ' in name.lower():
        return 'total_carbon_emissions'
    else:
        subtypes = [
            'gas fuel consumption', 'liquid fuel consumption', 'solid fuel consumption',
            'cement production', 'gas flaring', 'bunker fuels', 'per capita'
        ]
        for i in subtypes:
            if i in name.lower():
                return 'carbon_emissions_'+to_concept_id(i)
        # if nothing found, it should be a non measure concept.
        return to_concept_id(name)

In [30]:
nation_data.columns

Index(['nation', 'year',
       'total co2 emissions from fossil-fuels and cement production (thousand metric tons of c)',
       'emissions from solid fuel consumption',
       'emissions from liquid fuel consumption',
       'emissions from gas fuel consumption',
       'emissions from cement production', 'emissions from gas flaring',
       'per capita co2 emissions (metric tons of carbon)',
       'emissions from bunker fuels (not included in the totals)'],
      dtype='object')

In [31]:
list(map(get_concept_id, nation_data.columns))

['nation',
 'year',
 'total_carbon_emissions',
 'carbon_emissions_solid_fuel_consumption',
 'carbon_emissions_liquid_fuel_consumption',
 'carbon_emissions_gas_fuel_consumption',
 'carbon_emissions_cement_production',
 'carbon_emissions_gas_flaring',
 'carbon_emissions_per_capita',
 'carbon_emissions_bunker_fuels']

In [27]:
global_data.columns

Index(['year',
       'total carbon emissions from fossil fuel consumption and cement production (million metric tons of c)',
       'carbon emissions from gas fuel consumption',
       'carbon emissions from liquid fuel consumption',
       'carbon emissions from solid fuel consumption',
       'carbon emissions from cement production',
       'carbon emissions from gas flaring',
       'per capita carbon emissions (metric tons of carbon; after 1949 only)'],
      dtype='object')

In [32]:
list(map(get_concept_id, global_data.columns))

['year',
 'total_carbon_emissions',
 'carbon_emissions_gas_fuel_consumption',
 'carbon_emissions_liquid_fuel_consumption',
 'carbon_emissions_solid_fuel_consumption',
 'carbon_emissions_cement_production',
 'carbon_emissions_gas_flaring',
 'carbon_emissions_per_capita']

In [171]:
concept_discrete = ['year', 'nation', 'global', 'name', 'unit', 'description']

In [172]:
concept_all = np.r_[concept_discrete, list(map(get_concept_id, global_data.columns)), list(map(get_concept_id, nation_data.columns))]

In [173]:
concept_all = list(set(concept_all))

In [174]:
concept_all

['global',
 'carbon_emissions_gas_fuel_consumption',
 'carbon_emissions_per_capita',
 'carbon_emissions_solid_fuel_consumption',
 'nation',
 'description',
 'carbon_emissions_liquid_fuel_consumption',
 'total_carbon_emissions',
 'unit',
 'name',
 'year',
 'carbon_emissions_gas_flaring',
 'carbon_emissions_cement_production',
 'carbon_emissions_bunker_fuels']

In [175]:
cdf = pd.DataFrame(concept_all, columns=['concept'])

In [176]:
cdf

,concept
0,global
1,carbon_emissions_gas_fuel_consumption
2,carbon_emissions_per_capita
3,carbon_emissions_solid_fuel_consumption
4,nation
5,description
6,carbon_emissions_liquid_fuel_consumption
7,total_carbon_emissions
8,unit
9,name


In [53]:
def get_concept_name(concept):
    if concept.startswith('carbon_emissions'):
        n0 = 'Carbon Emissions'
        n1 = concept.replace('carbon_emissions_', '').replace('_', ' ').title()
        return n0 + ' From ' + n1
    else:
        return concept.replace('_', ' ').title()

In [177]:
cdf['name'] = cdf.concept.map(get_concept_name)

In [178]:
cdf

,concept,name
0,global,Global
1,carbon_emissions_gas_fuel_consumption,Carbon Emissions From Gas Fuel Consumption
2,carbon_emissions_per_capita,Carbon Emissions From Per Capita
3,carbon_emissions_solid_fuel_consumption,Carbon Emissions From Solid Fuel Consumption
4,nation,Nation
5,description,Description
6,carbon_emissions_liquid_fuel_consumption,Carbon Emissions From Liquid Fuel Consumption
7,total_carbon_emissions,Total Carbon Emissions
8,unit,Unit
9,name,Name


In [179]:
cdf['concept_type'] = cdf.concept.map(lambda x: 'measure' if 'carbon' in x else 'string')

cdf['unit'] = cdf.concept.map(lambda x: 'thousand metric tons' if 'carbon' in x else np.nan)

In [180]:
cdf = cdf.set_index('concept')
cdf.loc[['global', 'nation'], 'concept_type'] = 'entity_domain'

cdf.loc['total_carbon_emissions', 'description'] = 'Sum of fossil fuel consumption, cement production and gas flaring emissions'
cdf.loc['carbon_emissions_per_capita', 'unit'] = 'metric tonnes per person'
cdf.loc['year', 'concept_type'] = 'time'

cdf = cdf.reset_index()
cdf = cdf.sort_values(by='concept_type')

In [181]:
cdf

,concept,name,concept_type,unit,description
0,global,Global,entity_domain,NaN,NaN
4,nation,Nation,entity_domain,NaN,NaN
1,carbon_emissions_gas_fuel_consumption,Carbon Emissions From Gas Fuel Consumption,measure,thousand metric tons,NaN
2,carbon_emissions_per_capita,Carbon Emissions From Per Capita,measure,metric tonnes per person,NaN
3,carbon_emissions_solid_fuel_consumption,Carbon Emissions From Solid Fuel Consumption,measure,thousand metric tons,NaN
6,carbon_emissions_liquid_fuel_consumption,Carbon Emissions From Liquid Fuel Consumption,measure,thousand metric tons,NaN
7,total_carbon_emissions,Total Carbon Emissions,measure,thousand metric tons,"Sum of fossil fuel consumption, cement product..."
11,carbon_emissions_gas_flaring,Carbon Emissions From Gas Flaring,measure,thousand metric tons,NaN
12,carbon_emissions_cement_production,Carbon Emissions From Cement Production,measure,thousand metric tons,NaN
13,carbon_emissions_bunker_fuels,Carbon Emissions From Bunker Fuels,measure,thousand metric tons,NaN


In [182]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

# entities

In [67]:
nations_df = pd.DataFrame([nation_data.nation.map(to_concept_id).unique(), nation_data.nation.unique()])

In [70]:
nations_df = nations_df.T
nations_df.columns = ['nation', 'name']

In [74]:
nations_df.head()

,nation,name
0,afghanistan,AFGHANISTAN
1,albania,ALBANIA
2,algeria,ALGERIA
3,andorra,ANDORRA
4,angola,ANGOLA


In [77]:
nations_df.to_csv('../../ddf--entities--nation.csv', index=False)

In [78]:
global_ent = pd.DataFrame([['world', 'World']], columns=['global', 'name'])

In [80]:
global_ent.to_csv('../../ddf--entities--global.csv', index=False)

# datapoints

datapoints are in different units in nation file and global file. So we should make them same as stated in concepts files.

more details in https://github.com/open-numbers/ddf--cdiac--co2/issues/3

In [82]:
nation_data.columns = list(map(get_concept_id, nation_data.columns))

In [83]:
nation_data.nation = nation_data.nation.map(to_concept_id)

In [84]:
nation_data.head()

,nation,year,total_carbon_emissions,carbon_emissions_solid_fuel_consumption,carbon_emissions_liquid_fuel_consumption,carbon_emissions_gas_fuel_consumption,carbon_emissions_cement_production,carbon_emissions_gas_flaring,carbon_emissions_per_capita,carbon_emissions_bunker_fuels
3,afghanistan,1949,4.0,4.0,0.0,0.0,0.0,NaN,NaN,0.0
4,afghanistan,1950,23.0,6.0,18.0,0.0,0.0,0.0,0.0,0.0
5,afghanistan,1951,25.0,7.0,18.0,0.0,0.0,0.0,0.0,0.0
6,afghanistan,1952,25.0,9.0,17.0,0.0,0.0,0.0,0.0,0.0
7,afghanistan,1953,29.0,10.0,18.0,0.0,0.0,0.0,0.0,0.0


In [85]:
ndf = nation_data.set_index(['nation', 'year']).copy()

In [131]:
for col in ndf:
#     ndf[col] = ndf[col].map(format_float_digits)
    ndf[col].dropna().to_csv('../../ddf--datapoints--{}--by--nation--year.csv'.format(col), header=True)

In [114]:
global_data.columns = list(map(get_concept_id, global_data.columns))

In [115]:
global_data['global'] = 'world'

In [162]:
global_data.year = global_data.year.map(int)

In [221]:
# global data. global data is expressed in million tonnes, so we need to multiply them to make
# them same uint as nation data

gdf = global_data.set_index(['global', 'year']).copy()

for col in gdf:
    ser = gdf[col].map(float)  # not sure why some columns not reconized as float. fix those here.
    if 'per_capita' in col: # don't change per capita data
        ser.dropna().to_csv('../../ddf--datapoints--{}--by--global--year.csv'.format(col), header=True)
    else:  # multiply 1000
        (ser*1000).dropna().to_csv('../../ddf--datapoints--{}--by--global--year.csv'.format(col), header=True)

# Double Check: does sum of nation data equals global data?

In [284]:
# pick some random indicator

check = 'carbon_emissions_liquid_fuel_consumption'

In [285]:
nation_agg = ndf[check].groupby(level=1).sum()

In [292]:
a = nation_agg.iloc[120:150]

In [293]:
b = (gdf[check].map(float)*1000).groupby(level=1).sum().iloc[120:150]

In [294]:
pd.concat([a, b], axis=1)

,carbon_emissions_liquid_fuel_consumption,carbon_emissions_liquid_fuel_consumption
year,,
1871,878.0,1000.0
1872,990.0,1000.0
1873,1517.0,1000.0
1874,1662.0,1000.0
1875,1537.0,1000.0
1876,1708.0,1000.0
1877,2325.0,2000.0
1878,2633.0,2000.0
1879,3342.0,3000.0


**conclusion:** So we can see there is small differents in global and nation data. Not very big though

In [259]:
3065000.0 - 2685213.0

379787.0

In [260]:
379787.0 / 3065000

0.12391092985318107

In [154]:
# validation

In [187]:
!validate-ddf ../../ 

[
{"id":"INDEX_IS_NOT_FOUND","type":"Index is not found","path":"../../","tags":["WARNING","FILE_SYSTEM"]},

{}]

